In [1]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

'Trains a simple deep NN on the MNIST dataset.\nGets to 98.40% test accuracy after 20 epochs\n(there is *a lot* of margin for parameter tuning).\n2 seconds per epoch on a K520 GPU.\n'

keras에 필요한 class들을 import 합니다. keras는 보통 아래와 같이 다양한 요소들을 layer에서 상속된 클래스들로 표현이 되고, 이를 import해서 사용하는 형식으로 사용됩니다.

In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.utils import np_utils

Using TensorFlow backend.


사용할 파라미터를 설정합니다. 

In [1]:
batch_size = 128
nb_classes = 10
nb_epoch = 3

데이터를 읽어오는 부분입니다. keras 자체에 mnist 데이터를 읽어오는 함수가 내장되어 있고, 이를 사용해 받아올 수 있습니다.
load_data() 이후 데이터의 형태를 재구축(reshape)하고, 형태를 float으로 바꾸고(astype) 값을 0~255 범주에서 0.0~1.0 범주로 변경 (/=255)합니다.

In [4]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


keras의 np_utils.to_categorical은 integer의 list들을 nb_class 차원의 one-hot encoding으로 변환해주는 역할을 수행해 줍니다.

In [ ]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Multilayer perceptron 형태로 model을 구성합니다.

keras는 sequential과 functional 두가지 형식을 지원하고 아래는 sequential을 사용한 예시입니다.
queue 에 데이터를 넣듯이 model.add()를 통해 layer들을 넣을 수 있습니다. 아래의 예시는 3층으로 쌓여있고, 중간중간 Dropout이 들어가고, activation으로 relu를 사용하는 것을 보여줍니다.

model.summary()를 실행하면 모델에 대한 구성을 출력하여 줍니다.

In [5]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 512)           401920      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 512)           0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 512)           0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 512)           262656      dropout_1[0][0]                  
___________________________________________________________________________________________

loss를 multiclass 용 loss인 categorical_crossentropy로 정하고, optimizer로는 RMSprop를, 평가 metric은 accuracy로 설정하도록 합니다.

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

아래의 model.fit을 통해 실제 학습을 수행합니다.

In [7]:
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 9s - loss: 0.2442 - acc: 0.9244 - val_loss: 0.1189 - val_acc: 0.9630

학습된 모델을 기반으로 test셋의 성능을 측정하여봅니다.

In [8]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0979186110775
Test accuracy: 0.9851
